# Preparação dos Dados

> Adicionar aspas



## Etapas que serão realizadas no processo

- Gerar Metadados da ABT (Tabela Analítica de Modelagem)
- Tratamento de missing (nulos)
- Tratamento de categóricas de alta cardinalidade (LabelEncoder)
- Tratamento de categóricas de baixa cardinalidade (OneHotEncoder)
- Aplicar normalização a toda tabela de modelagem (ABT)
- Gerar artefatos para implantação do data prep realizado

In [ ]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
  from google.colab import drive
  drive.mount('/content/drive')

  # Google Drive
  dir_base = "/content/drive/MyDrive/jupyter/pcd_porto_seguro_safe_driver_prediction/data/"
  # raiz
  dir_base_root = "/content/drive/MyDrive/jupyter/pcd_porto_seguro_safe_driver_prediction/"
  import sys
  # caution: path[0] is reserved for script path (or '' in REPL)
  sys.path.append(dir_base_root)

else:
  # Diretorio da base dos dados
  # Local PC
  dir_base = "data/"


Mounted at /content/drive


In [ ]:
# Funções utilizadas no projeto
#import data_science_lib as dslib
import pod_academy_functions as pod


# Bibliotecas utilizadas durante o projeto
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

-----------------------------------------------------
------------ PoD Academy Functions ------------------
Só Alegria!
Você está usando o pacote de funções da PoD Academy
Turma: 2023
Autor: PoD Academy
Professor: Bruno Jardim
Versão: 1.0.0
Data: 08/10/2023
-----------------------------------------------------


In [ ]:
# Ajuste para o pandas exibir todas as colunas
pd.set_option('display.max_columns', None)

In [ ]:
# Leitura dos dados de TREINO disponibilizados pela Porto Seguro
df_train_00 = pd.read_csv(dir_base + "train.csv")

In [ ]:
# Checagem do número de linhas e colunas/variaveis
df_train_00.shape

(595212, 59)

In [ ]:
# Cria uma amostra aleatória de itens da base Treino.
# Você pode usar random_state para garantir reprodutibilidade.
df_train_00_sample = df_train_00.sample(n=20000, random_state=6)
# Checagem do volume do Dataframe
df_train_00_sample.shape

(20000, 59)

In [ ]:
# Leitura dos dados de TESTE disponibilizados pela Porto Seguro
df_teste_00 = pd.read_csv(dir_base + "test.csv")
# Checagem do volume do Dataframe
df_teste_00.shape

(892816, 58)

### Separação dos dados para garantir validação cruzada Holdout 60/40

In [ ]:
# Separa a base do DataFrame Treino em treino = 60% e test = 40%
train, test = train_test_split(df_train_00, test_size=0.4, random_state=6)

In [ ]:
# Checagem dos dados
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
510263,1274939,0,3,1,8,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.3,0.0,0.982662,11,1,-1,2,-1,11,1,1,2,1,14,0,0.374166,0.674659,0.430232,2.828427,0.3,0.6,0.6,2,0,7,2,11,2,10,3,1,3,7,0,1,0,0,0,0
566227,1415654,0,0,1,5,0,0,1,0,0,0,0,0,0,0,0,11,1,0,0,0.6,0.0,0.680533,6,1,-1,0,0,7,1,1,0,1,96,3,0.446878,0.587842,0.430116,1.000000,0.8,0.0,0.9,1,1,6,4,10,4,6,5,0,2,6,0,0,1,1,1,0
154123,385376,1,1,2,2,0,0,1,0,0,0,0,0,0,0,0,8,0,1,0,0.9,0.3,0.572822,11,1,-1,0,-1,0,1,1,2,1,87,3,0.316228,0.698833,0.361525,3.162278,0.9,0.8,0.4,3,2,7,1,9,0,8,9,0,3,11,0,1,0,1,1,0
500653,1251174,0,5,1,4,0,0,0,1,0,0,0,0,0,0,0,11,1,0,0,0.9,0.6,0.675000,11,0,1,1,0,2,-1,1,2,1,104,2,0.447214,1.774253,-1.000000,3.464102,0.5,0.5,0.8,4,1,9,1,10,3,10,2,2,4,10,0,0,0,0,1,0
362160,904785,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.9,1.3,1.216296,11,0,-1,0,-1,0,1,1,2,1,46,3,0.400000,0.786718,0.366333,3.316625,0.7,0.7,0.7,3,4,7,2,8,4,5,6,1,3,9,0,1,1,0,0,0


### Tratamento dos Dados
#### Tratamento de Missing (nulos)
Conforme visto na etapa de entendimento do negócio, os valores de -1 indicam que o recurso estava faltando na observação (Nulo).


In [ ]:
# Substituir -1 por 'missing' em todo o dataframe de treino
# Gerar um novo dataframe a partir dessa mudança com base no original
# Copia o original
df_train_01 = train.copy()

In [ ]:
# Aplica o replace em todo o dataframe
df_train_01.replace(-1, np.nan, inplace=True)

In [ ]:
df_train_01.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
510263,1274939,0,3,1.0,8,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.3,0.0,0.982662,11.0,1.0,NaN,2,NaN,11,1.0,1,2.0,1,14,0.0,0.374166,0.674659,0.430232,2.828427,0.3,0.6,0.6,2,0,7,2,11,2,10,3,1,3,7,0,1,0,0,0,0
566227,1415654,0,0,1.0,5,0.0,0.0,1,0,0,0,0,0,0,0,0,11,1,0,0,0.6,0.0,0.680533,6.0,1.0,NaN,0,0.0,7,1.0,1,0.0,1,96,3.0,0.446878,0.587842,0.430116,1.000000,0.8,0.0,0.9,1,1,6,4,10,4,6,5,0,2,6,0,0,1,1,1,0
154123,385376,1,1,2.0,2,0.0,0.0,1,0,0,0,0,0,0,0,0,8,0,1,0,0.9,0.3,0.572822,11.0,1.0,NaN,0,NaN,0,1.0,1,2.0,1,87,3.0,0.316228,0.698833,0.361525,3.162278,0.9,0.8,0.4,3,2,7,1,9,0,8,9,0,3,11,0,1,0,1,1,0
500653,1251174,0,5,1.0,4,0.0,0.0,0,1,0,0,0,0,0,0,0,11,1,0,0,0.9,0.6,0.675000,11.0,0.0,1.0,1,0.0,2,NaN,1,2.0,1,104,2.0,0.447214,1.774253,NaN,3.464102,0.5,0.5,0.8,4,1,9,1,10,3,10,2,2,4,10,0,0,0,0,1,0
362160,904785,0,0,1.0,1,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.9,1.3,1.216296,11.0,0.0,NaN,0,NaN,0,1.0,1,2.0,1,46,3.0,0.400000,0.786718,0.366333,3.316625,0.7,0.7,0.7,3,4,7,2,8,4,5,6,1,3,9,0,1,1,0,0,0


In [ ]:
metadados = pod.pod_academy_generate_metadata(df_train_01,
    ids = ['id'],
    targets = ['target'],
    orderby = 'PC_NULOS'
)

In [ ]:
# Visualização dos metadados gerados com base no dataframe treino
# Essas dados ajudarão na tomada de decisão do que será feito com as variaveis
metadados

,FEATURE,USO_FEATURE,QT_NULOS,PC_NULOS,CARDINALIDADE,TIPO_FEATURE
0,ps_car_03_cat,Explicativa,246862,69.12,2,float64
1,ps_car_05_cat,Explicativa,160003,44.80,2,float64
2,ps_reg_03,Explicativa,64556,18.08,4940,float64
3,ps_car_14,Explicativa,25597,7.17,820,float64
4,ps_car_07_cat,Explicativa,6845,1.92,2,float64
5,ps_ind_05_cat,Explicativa,3483,0.98,7,float64
6,ps_car_09_cat,Explicativa,321,0.09,5,float64
7,ps_ind_02_cat,Explicativa,142,0.04,4,float64
8,ps_ind_04_cat,Explicativa,54,0.02,2,float64
9,ps_car_01_cat,Explicativa,68,0.02,12,float64


#### Exclusão de variaveis com alta quantidade de nulos
- Com base no metadados, podemos ver que a variavel ps_car_03_cat	do tipo Explicativa	possui 69.12% de nulos
- Com base nessa informação foi definido um corte onde variaveis acima de 68% de nulo serão excluídas

In [ ]:
# Valor para corte de variaveis nulas
missing_cutoff = 68
# Seleciona as variaveis com valores acima ou igual ao missing_cutoff definido
drop_vars_nulls = metadados[(metadados['PC_NULOS']>= missing_cutoff)]

In [ ]:
# Cria uma lista com as variaveis que será utilizada para remover
list_drop_vars = list(drop_vars_nulls.FEATURE.values)

In [ ]:
print('Lista de variáveis que serão excluídas por alto percentual de nulos:', list_drop_vars)
# Processo de retirada das variaveis com alto percentual de nulos
# Criado um novo dataframe para receber os dados já atualizados e sem a variavel
df_train_02 = df_train_01.drop(axis=1, columns=list_drop_vars)

Lista de variáveis que serão excluídas por alto percentual de nulos: ['ps_car_03_cat']


In [ ]:
#Checagem dos dados
df_train_02.shape

(357127, 58)

In [ ]:
# Salva a lista de variaveis excluidas em um arquivo pkl para implantação
with open(dir_base + "prd_list_drop_vars.pkl", 'wb') as f:
    pickle.dump(list_drop_vars, f)

In [ ]:
# Remove o ID e o Target do dataframe que esta sendo tratado
df_train_02 = df_train_02.drop(axis=1, columns=['id', 'target'])

In [ ]:
df_train_02.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
510263,3,1.0,8,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.3,0.0,0.982662,11.0,1.0,2,NaN,11,1.0,1,2.0,1,14,0.0,0.374166,0.674659,0.430232,2.828427,0.3,0.6,0.6,2,0,7,2,11,2,10,3,1,3,7,0,1,0,0,0,0
566227,0,1.0,5,0.0,0.0,1,0,0,0,0,0,0,0,0,11,1,0,0,0.6,0.0,0.680533,6.0,1.0,0,0.0,7,1.0,1,0.0,1,96,3.0,0.446878,0.587842,0.430116,1.000000,0.8,0.0,0.9,1,1,6,4,10,4,6,5,0,2,6,0,0,1,1,1,0
154123,1,2.0,2,0.0,0.0,1,0,0,0,0,0,0,0,0,8,0,1,0,0.9,0.3,0.572822,11.0,1.0,0,NaN,0,1.0,1,2.0,1,87,3.0,0.316228,0.698833,0.361525,3.162278,0.9,0.8,0.4,3,2,7,1,9,0,8,9,0,3,11,0,1,0,1,1,0
500653,5,1.0,4,0.0,0.0,0,1,0,0,0,0,0,0,0,11,1,0,0,0.9,0.6,0.675000,11.0,0.0,1,0.0,2,NaN,1,2.0,1,104,2.0,0.447214,1.774253,NaN,3.464102,0.5,0.5,0.8,4,1,9,1,10,3,10,2,2,4,10,0,0,0,0,1,0
362160,0,1.0,1,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.9,1.3,1.216296,11.0,0.0,0,NaN,0,1.0,1,2.0,1,46,3.0,0.400000,0.786718,0.366333,3.316625,0.7,0.7,0.7,3,4,7,2,8,4,5,6,1,3,9,0,1,1,0,0,0


### Tratamento de Nulos (Missing)

Nesse processo de preparação dos dados de treino foi a imputação das variáveis categóricas e numéricas:
- Variáveis Categóricas: Realizado processamento de imputação para o valor "MISS_VERIFICAR"
- Variáveis Numéricas: Ajustado para substituir os missing pela média
- Apenas ficar atento com outliers e checar a base, dependendo da situação testar com a imputação pelo valor da medianaplicação).

In [ ]:
# Essa etapa com o uso da função realiza as seguintes ações:
# - Separa as variáveis númericas e preenche com a média os valores nulos
# - Separa as variáveis categóricas e preenche com MISS_VERIFICAR
# - Retorna o dataframe atualizado e a lista com as médias de cada variável
df_train_03, means = pod.pod_custom_fillna(df_train_02)

In [ ]:
# Visualiza dataframe atualizado
df_train_03.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
510263,3,1.0,8,0.0,0.0,0,0,1,0,0,0,0,0,0,1,0,1,0,0.3,0.0,0.982662,11.0,1.0,2,0.526181,11,1.000000,1,2.0,1,14,0.0,0.374166,0.674659,0.430232,2.828427,0.3,0.6,0.6,2,0,7,2,11,2,10,3,1,3,7,0,1,0,0,0,0
566227,0,1.0,5,0.0,0.0,1,0,0,0,0,0,0,0,0,11,1,0,0,0.6,0.0,0.680533,6.0,1.0,0,0.000000,7,1.000000,1,0.0,1,96,3.0,0.446878,0.587842,0.430116,1.000000,0.8,0.0,0.9,1,1,6,4,10,4,6,5,0,2,6,0,0,1,1,1,0
154123,1,2.0,2,0.0,0.0,1,0,0,0,0,0,0,0,0,8,0,1,0,0.9,0.3,0.572822,11.0,1.0,0,0.526181,0,1.000000,1,2.0,1,87,3.0,0.316228,0.698833,0.361525,3.162278,0.9,0.8,0.4,3,2,7,1,9,0,8,9,0,3,11,0,1,0,1,1,0
500653,5,1.0,4,0.0,0.0,0,1,0,0,0,0,0,0,0,11,1,0,0,0.9,0.6,0.675000,11.0,0.0,1,0.000000,2,0.947985,1,2.0,1,104,2.0,0.447214,1.774253,0.374586,3.464102,0.5,0.5,0.8,4,1,9,1,10,3,10,2,2,4,10,0,0,0,0,1,0
362160,0,1.0,1,0.0,0.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.9,1.3,1.216296,11.0,0.0,0,0.526181,0,1.000000,1,2.0,1,46,3.0,0.400000,0.786718,0.366333,3.316625,0.7,0.7,0.7,3,4,7,2,8,4,5,6,1,3,9,0,1,1,0,0,0


In [ ]:
# Salva a lista de variaveis e o valor médio imputado nos nulos em um arquivo pkl para implantação
with open(dir_base + 'prd_fillna.pkl', 'wb') as f:
    pickle.dump(means, f)

In [ ]:
# Carrega o arquivo pkl com a lista de variaveis e o valor médio imputado nos nulos e coloca em uma lista

with open(dir_base + 'prd_fillna.pkl', 'rb') as f:
    loaded_means_vars = pickle.load(f)

# Visualiza a lista carregada
loaded_means_vars

{'ps_ind_01': 1.9008083958927777,
 'ps_ind_02_cat': 1.3609759513705058,
 'ps_ind_03': 4.429077051021065,
 'ps_ind_04_cat': 0.4173684372663293,
 'ps_ind_05_cat': 0.4196536630057346,
 'ps_ind_06_bin': 0.3937982846438382,
 'ps_ind_07_bin': 0.2562757786445718,
 'ps_ind_08_bin': 0.16414608808631104,
 'ps_ind_09_bin': 0.18577984862527896,
 'ps_ind_10_bin': 0.00037801678394520715,
 'ps_ind_11_bin': 0.0016996754655906722,
 'ps_ind_12_bin': 0.009383216614817698,
 'ps_ind_13_bin': 0.0009128405301195373,
 'ps_ind_14': 0.012373749394473114,
 'ps_ind_15': 7.3024610292697,
 'ps_ind_16_bin': 0.6610001484065893,
 'ps_ind_17_bin': 0.12081136402456269,
 'ps_ind_18_bin': 0.15338521030333746,
 'ps_reg_01': 0.6114284274221776,
 'ps_reg_02': 0.43946747235577255,
 'ps_reg_03': 0.8945804378201793,
 'ps_car_01_cat': 8.296345982036582,
 'ps_car_02_cat': 0.8302751137556877,
 'ps_car_04_cat': 0.7219896563407415,
 'ps_car_05_cat': 0.5261814898236643,
 'ps_car_06_cat': 6.550532443640497,
 'ps_car_07_cat': 0.9479847

In [ ]:
# Essa etapa com o uso da função realiza as seguintes ações:
# - Substitui valor -1 por nulos em todas as variáveis
# - Separa as variáveis númericas e preenche com a média salva da lista os valores nulos (utilizando assim o mesmo valor de treino)
# - Separa as variáveis categóricas e preenche com MISS_VERIFICAR
# - Retorna o dataframe atualizado com os valores imputados
test_prod = pod.pod_custom_fillna_prod(test, loaded_means_vars)

# Checagem do volume de linhas e colunas
test_prod.shape

(238085, 59)

In [ ]:
# Checagem dos dados
test_prod.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
274442,685895,0,1,3.0,5,1.0,3.0,0,0,1,0,0,0,0,0,0,13,1,0,0,0.4,0.2,0.737818,11.0,1.0,NaN,0,0.526181,0,1.0,1,0.0,1,87,3.0,0.316228,0.743838,0.361525,3.316625,0.1,0.1,0.0,1,3,7,3,11,4,14,8,1,0,5,0,1,0,0,0,0
582075,1455073,0,0,1.0,7,0.0,0.0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.6,1.2,1.190851,11.0,1.0,0.0,9,0.000000,15,1.0,1,2.0,1,41,2.0,0.547723,1.212529,0.416533,2.645751,0.4,0.4,0.4,2,4,4,1,11,1,7,4,5,0,10,0,1,1,0,1,0
110994,277852,0,2,1.0,11,0.0,0.0,0,0,0,1,0,0,0,0,0,12,0,0,1,0.1,0.2,0.894580,4.0,1.0,NaN,0,1.000000,0,1.0,1,0.0,1,66,1.0,0.400000,0.763920,0.379473,3.000000,0.0,0.8,0.7,3,2,8,2,11,5,7,2,2,2,4,0,1,1,0,0,0
291457,728292,0,0,1.0,0,1.0,0.0,0,0,1,0,0,0,0,0,0,2,1,0,0,0.6,0.2,0.715017,11.0,1.0,NaN,0,0.526181,1,1.0,1,3.0,1,64,3.0,0.316228,0.686312,0.361939,3.316625,0.9,0.3,0.8,2,3,8,1,11,1,17,6,2,2,5,0,1,0,0,1,0
3095,7922,0,0,1.0,2,1.0,0.0,0,1,0,0,0,0,0,0,0,13,1,0,0,0.1,0.2,0.894580,6.0,1.0,NaN,0,1.000000,1,1.0,1,0.0,1,10,2.0,0.316228,0.700074,0.381445,3.162278,0.4,0.5,0.6,1,4,7,3,9,2,12,6,0,3,5,0,0,1,1,0,0


#### Tratamento de variáveis categóricas de alta cardinalidade (Utilização da técnica LabelEncoder)

In [ ]:
metadados

,FEATURE,USO_FEATURE,QT_NULOS,PC_NULOS,CARDINALIDADE,TIPO_FEATURE
0,ps_car_03_cat,Explicativa,246862,69.12,2,float64
1,ps_car_05_cat,Explicativa,160003,44.80,2,float64
2,ps_reg_03,Explicativa,64556,18.08,4940,float64
3,ps_car_14,Explicativa,25597,7.17,820,float64
4,ps_car_07_cat,Explicativa,6845,1.92,2,float64
5,ps_ind_05_cat,Explicativa,3483,0.98,7,float64
6,ps_car_09_cat,Explicativa,321,0.09,5,float64
7,ps_ind_02_cat,Explicativa,142,0.04,4,float64
8,ps_ind_04_cat,Explicativa,54,0.02,2,float64
9,ps_car_01_cat,Explicativa,68,0.02,12,float64


#### Importante
A base de dados fornecida pela Porto Seguro aparentemente já teve a técnica de LabelEncoder ou outra aplicada. No caso não existe nenhuma variavel do tipo object para ser aplicado e já se encontram com os valores preenchidos.

In [ ]:
# Nessa etapa esta sendo selecionada a lista de variaveis categoricas que precisam ser aplicado o LabelEncoder
# Foi definido um valor de cutoff onde as variaveis que tiverem cardinalidade maior e também forem do tipo object
# Após separar a lista das colunas tanto do dataframe treino e da lista de labelencoder
card_cutoff = 17

#df_cat_labelenc = metadados[(metadados['CARDINALIDADE'] > card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
df_cat_labelenc = metadados[(metadados['CARDINALIDADE'] > card_cutoff) & (metadados['FEATURE'].str.contains('_cat'))]
list_vars_abt = list(df_train_03.columns)
list_lblenc = list(df_cat_labelenc.FEATURE.values)

for item in list_drop_vars:
    if item in list_lblenc:
        list_lblenc.remove(item)

print('Lista de variáveis selecionadas para aplicar Label Encoder', list_lblenc)


Lista de variáveis selecionadas para aplicar Label Encoder ['ps_car_11_cat', 'ps_car_06_cat']


In [ ]:
# Importe da biblioteca para realizar o LabelEncoder
from sklearn.preprocessing import LabelEncoder

encoders = {}

for column in list_lblenc:
    encoder = LabelEncoder()
    df_train_03[column] = encoder.fit_transform(df_train_03[column])

    # Armazena o Encoder gerado para a coluna atual em um dicionário
    encoders[column] = encoder

# Gera o dicionário de encoders e a lista de colunas
data_to_serialize = {
    'encoders': encoders,
    'columns': list_lblenc
}

# Salva arquivo pkl para que possa ser reutilizado
with open(dir_base + 'prd_labelencoder.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)


In [ ]:
# Carrega os encoders e a lista de colunas para serem aplicados na base de test
# Conforme já explicado não deve ver mudanças devido a base já ter vindo com toda a transformação
with open(dir_base + 'prd_labelencoder.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

for column in loaded_columns:
    if column in loaded_encoders:
        # Aplica a transformação para a coluna com o valor do encoder carregado
        test[column] = loaded_encoders[column].transform(test[column])

In [ ]:
# Visualização dos dados
test.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
274442,685895,0,1,3.0,5,1.0,3.0,0,0,1,0,0,0,0,0,0,13,1,0,0,0.4,0.2,0.737818,11.0,1.0,NaN,0,0.526181,0,1.0,1,0.0,1,86,3.0,0.316228,0.743838,0.361525,3.316625,0.1,0.1,0.0,1,3,7,3,11,4,14,8,1,0,5,0,1,0,0,0,0
582075,1455073,0,0,1.0,7,0.0,0.0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.6,1.2,1.190851,11.0,1.0,0.0,9,0.000000,15,1.0,1,2.0,1,40,2.0,0.547723,1.212529,0.416533,2.645751,0.4,0.4,0.4,2,4,4,1,11,1,7,4,5,0,10,0,1,1,0,1,0
110994,277852,0,2,1.0,11,0.0,0.0,0,0,0,1,0,0,0,0,0,12,0,0,1,0.1,0.2,0.894580,4.0,1.0,NaN,0,1.000000,0,1.0,1,0.0,1,65,1.0,0.400000,0.763920,0.379473,3.000000,0.0,0.8,0.7,3,2,8,2,11,5,7,2,2,2,4,0,1,1,0,0,0
291457,728292,0,0,1.0,0,1.0,0.0,0,0,1,0,0,0,0,0,0,2,1,0,0,0.6,0.2,0.715017,11.0,1.0,NaN,0,0.526181,1,1.0,1,3.0,1,63,3.0,0.316228,0.686312,0.361939,3.316625,0.9,0.3,0.8,2,3,8,1,11,1,17,6,2,2,5,0,1,0,0,1,0
3095,7922,0,0,1.0,2,1.0,0.0,0,1,0,0,0,0,0,0,0,13,1,0,0,0.1,0.2,0.894580,6.0,1.0,NaN,0,1.000000,1,1.0,1,0.0,1,9,2.0,0.316228,0.700074,0.381445,3.162278,0.4,0.5,0.6,1,4,7,3,9,2,12,6,0,3,5,0,0,1,1,0,0


#### Tratamento de variáveis categóricas de baixa cardinalidade

In [ ]:
# Importação da biblioteca para tratamento com OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
card_cutoff = 2
#df_categ_onehot = metadados[(metadados['CARDINALIDADE'] <= card_cutoff) & (metadados['TIPO_FEATURE'] == 'object')]
df_categ_onehot = metadados[(metadados['CARDINALIDADE'] <= card_cutoff) & (metadados['FEATURE'].str.contains('_cat'))]
list_ohc = list(df_categ_onehot.FEATURE.values)

for item in list_drop_vars:
    if item in list_ohc:
        list_ohc.remove(item)

print('Lista de variáveis para aplicar o OneHot Encoding', list_ohc)

# Instanciando OneHotEncoder para uso
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplica o encoder nas colunas
encoded_data = encoder.fit_transform(df_train_03[list_ohc])
encoded_cols = encoder.get_feature_names_out(list_ohc)

#cria um novo dataframe somente com os dados encodados
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index = df_train_03.index)

# Concatena os dataframe de treino e encodado e remove as colunas que sofreram encoder
df_train_03 = pd.concat([df_train_03.drop(list_ohc, axis=1), encoded_df], axis=1)

# Salva o encoder e a lista de colunas em um arquivo .pkl para reaproveitamento
data_to_serialize = {
    'encoder': encoder,
    'columns': list_ohc
}

# Salva o arquivo pkl
with open(dir_base + 'prd_onehotencoder.pkl', 'wb') as f:
    pickle.dump(data_to_serialize, f)


Lista de variáveis para aplicar o OneHot Encoding ['ps_car_05_cat', 'ps_car_07_cat', 'ps_ind_04_cat', 'ps_car_02_cat', 'ps_car_08_cat']


In [ ]:
# Carrega o encoder a lista de de colunas para ser aplicado na base de test
with open(dir_base + 'prd_onehotencoder.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# carrega o encoder e as colunas em suas variaveis para uso
loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Suponha test_df como sua base de teste
# Aplica o encoder nas colunas
encoded_data_test = loaded_encoder.transform(test[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)

#Cria dataframe com colunas encodadas
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=test.index)
# Concatena os dataframe de test e o encodado e remove as colunas que sofreram encoder
test = pd.concat([test.drop(loaded_columns, axis=1), encoded_df_test], axis=1)


In [ ]:
test.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_03_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1
274442,685895,0,1,3.0,5,3.0,0,0,1,0,0,0,0,0,0,13,1,0,0,0.4,0.2,0.737818,11.0,NaN,0,0,0.0,1,86,3.0,0.316228,0.743838,0.361525,3.316625,0.1,0.1,0.0,1,3,7,3,11,4,14,8,1,0,5,0,1,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
582075,1455073,0,0,1.0,7,0.0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.6,1.2,1.190851,11.0,0.0,9,15,2.0,1,40,2.0,0.547723,1.212529,0.416533,2.645751,0.4,0.4,0.4,2,4,4,1,11,1,7,4,5,0,10,0,1,1,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
110994,277852,0,2,1.0,11,0.0,0,0,0,1,0,0,0,0,0,12,0,0,1,0.1,0.2,0.894580,4.0,NaN,0,0,0.0,1,65,1.0,0.400000,0.763920,0.379473,3.000000,0.0,0.8,0.7,3,2,8,2,11,5,7,2,2,2,4,0,1,1,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
291457,728292,0,0,1.0,0,0.0,0,0,1,0,0,0,0,0,0,2,1,0,0,0.6,0.2,0.715017,11.0,NaN,0,1,3.0,1,63,3.0,0.316228,0.686312,0.361939,3.316625,0.9,0.3,0.8,2,3,8,1,11,1,17,6,2,2,5,0,1,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3095,7922,0,0,1.0,2,0.0,0,1,0,0,0,0,0,0,0,13,1,0,0,0.1,0.2,0.894580,6.0,NaN,0,1,0.0,1,9,2.0,0.316228,0.700074,0.381445,3.162278,0.4,0.5,0.6,1,4,7,3,9,2,12,6,0,3,5,0,0,1,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


#### Aplicar normalização a toda a tabela de modelagem tratada até o momento

In [ ]:
# Importe da biblioteca para processamento da normalização
from sklearn.preprocessing import StandardScaler

In [ ]:
# Remover as colunas que não devem ser normalizadas
# Utilizar o metadados para selecionar as colunas com base nas informações
# cria um Dataframe com os metadados de ID e Target
df_id_target = metadados[(metadados['USO_FEATURE']=='ID') | (metadados['USO_FEATURE']=='Target')]

# cria uma lista com os nomes das features
list_id_target = list(df_id_target.FEATURE.values)

print('Lista de IDs e Target', list_id_target)

Lista de IDs e Target ['id', 'target']


In [ ]:
# Instancia o StandardScaler para aplicar normalização no Dataframe
scaler = StandardScaler()

# Etapa de padronização da base de treino
df_train_03_scaled = scaler.fit_transform(df_train_03)

# Gera o novo dataframe com os valores normalizados
df_train_04 = pd.DataFrame(df_train_03_scaled, columns=df_train_03.columns, index=df_train_03.index)

In [ ]:
# Salva o scaler em um arquivo .pkl
with open(dir_base + 'prd_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
# Visão geral dos dados normalizados
df_train_04.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1
510263,0.553925,-0.543007,1.321585,-0.311952,-0.805988,-0.587013,2.256576,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.777632,-1.396372,2.697660,-0.425646,-1.083208,-1.086909,0.281831,1.078377,0.594728,0.808380,0.685453,0.086523,-1.459265,-2.816553,-0.096613,-0.616369,1.269533e+00,-0.326028,-0.521373,0.523309,0.522092,-0.331863,-1.660097,-0.516858,-0.712243,1.213911,-0.272846,0.537574,-1.047586,-0.366532,0.073825,-0.198007,-0.374444,0.769761,-1.115433,-0.635384,-0.732096,-0.425252,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929
566227,-0.957891,-0.543007,0.211296,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.042904,0.716142,-0.370692,-0.425646,-0.039750,-1.086909,-0.684878,-0.915918,-0.335981,0.081659,-1.362990,0.086523,1.024187,0.785650,1.153130,-1.003274,1.266881e+00,-2.828849,1.219037,-1.566526,1.566821,-1.228105,-0.779611,-1.266900,0.702467,0.528059,1.330772,-0.837398,-0.189968,-1.197395,-0.515582,-0.562370,-0.374444,-1.299104,0.896513,1.573851,1.365942,-0.425252,-0.900937,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929
154123,-0.453952,0.961267,-0.898992,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,0.196743,-1.396372,2.697660,-0.425646,1.003708,-0.344937,-1.029515,1.078377,-0.335981,-1.190102,0.685453,0.086523,0.751613,0.785650,-1.092421,-0.508637,-2.980005e-01,0.130960,1.567119,1.219920,-0.174393,0.564379,0.100875,-0.516858,-1.419598,-0.157794,-1.876463,-0.149912,1.525267,-1.197395,0.073825,1.259443,-0.374444,0.769761,-1.115433,1.573851,1.365942,-0.425252,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929
500653,1.561802,-0.543007,-0.158800,-0.311952,-0.805988,1.703539,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.042904,0.716142,-0.370692,-0.425646,1.003708,0.397036,-0.702581,1.078377,0.129373,-0.826741,0.685453,0.086523,1.266475,-0.415085,1.158897,4.284008,1.266456e-15,0.544108,0.174791,0.175003,1.218578,1.460621,-0.779611,0.983227,-1.419598,0.528059,0.528963,0.537574,-1.476394,0.464332,0.663231,0.895081,-0.374444,-1.299104,-1.115433,-0.635384,1.365942,-0.425252,-0.900937,-0.63978,7.153559,-3.639785,-0.012298,-0.846265,-0.002366,-2.211725,0.448929
362160,-0.957891,-0.543007,-1.269089,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.777632,-1.396372,-0.370692,-0.425646,1.003708,2.128305,1.029377,1.078377,-0.335981,-1.190102,0.685453,0.086523,-0.490113,0.785650,0.347414,-0.116975,-1.882943e-01,0.342236,0.870955,0.871614,0.870335,0.564379,1.861846,-0.516858,-0.712243,-0.843646,1.330772,-1.181140,0.238841,-0.366532,0.073825,0.530718,-0.374444,0.769761,0.896513,-0.635384,-0.732096,-0.425252,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,-2.211725,0.448929


In [ ]:
#Colunas que foram retiradas da tabela:
# ID e target para não serem utilizadas na normalizada
# ps_car_03_cat por conter mais de 68% de valores nulos
list_columns_drop = ['id', 'target', 'ps_car_03_cat']

# Cria um dataframe de test extra sem as colunas para que possa ser feito a normalização nele
df_test_aux = test.drop(axis=1, columns=list_columns_drop)

In [ ]:
# Visualização dos dados
df_test_aux.head(5)

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1
274442,1,3.0,5,3.0,0,0,1,0,0,0,0,0,0,13,1,0,0,0.4,0.2,0.737818,11.0,0,0,0.0,1,86,3.0,0.316228,0.743838,0.361525,3.316625,0.1,0.1,0.0,1,3,7,3,11,4,14,8,1,0,5,0,1,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
582075,0,1.0,7,0.0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.6,1.2,1.190851,11.0,9,15,2.0,1,40,2.0,0.547723,1.212529,0.416533,2.645751,0.4,0.4,0.4,2,4,4,1,11,1,7,4,5,0,10,0,1,1,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
110994,2,1.0,11,0.0,0,0,0,1,0,0,0,0,0,12,0,0,1,0.1,0.2,0.894580,4.0,0,0,0.0,1,65,1.0,0.400000,0.763920,0.379473,3.000000,0.0,0.8,0.7,3,2,8,2,11,5,7,2,2,2,4,0,1,1,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
291457,0,1.0,0,0.0,0,0,1,0,0,0,0,0,0,2,1,0,0,0.6,0.2,0.715017,11.0,0,1,3.0,1,63,3.0,0.316228,0.686312,0.361939,3.316625,0.9,0.3,0.8,2,3,8,1,11,1,17,6,2,2,5,0,1,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3095,0,1.0,2,0.0,0,1,0,0,0,0,0,0,0,13,1,0,0,0.1,0.2,0.894580,6.0,0,1,0.0,1,9,2.0,0.316228,0.700074,0.381445,3.162278,0.4,0.5,0.6,1,4,7,3,9,2,12,6,0,3,5,0,0,1,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


In [ ]:
# Carrega o arquivo com as informações do scaler para aplicar na tabela de test e manter o mesmo padrão
with open(dir_base + 'prd_scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

In [ ]:
# Suponha test_df como sua base de teste
# Aplica a normalização no dataframe auxiliar
test_df_scaled = loaded_scaler.transform(df_test_aux)

# Cria o dataframe com os valores normalizados
test_df = pd.DataFrame(test_df_scaled, columns=df_test_aux.columns, index=df_test_aux.index)

# Visuallização dos dados no df normalizado
test_df.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1
274442,-0.453952,2.465540,0.211296,1.918116,-0.805988,-0.587013,2.256576,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.607012,0.716142,-0.370692,-0.425646,-0.735389,-0.592261,-5.015862e-01,1.078377,-0.335981,-1.190102,-1.362990,0.086523,0.751613,0.785650,-1.092421,-0.308070,-0.298001,0.342236,-1.217537,-1.218220,-1.567364,-1.228105,0.981360,-0.516858,-0.004888,1.213911,1.330772,1.912545,1.096458,-0.366532,-1.694394,-0.926732,-0.374444,0.769761,-1.115433,-0.635384,-0.732096,-0.425252,1.109956,-0.639780,-0.139791,0.274741,-0.012298,1.181663,-0.002366,0.452136,0.448929
582075,-0.957891,-0.543007,0.951489,-0.311952,-0.805988,1.703539,-0.443149,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.213525,-1.396372,-0.370692,2.349369,-0.039750,1.880981,9.479614e-01,1.078377,3.852213,1.535100,0.685453,0.086523,-0.641543,-0.415085,2.886394,1.780666,0.956994,-0.576081,-0.173291,-0.173303,-0.174393,-0.331863,1.861846,-2.766985,-1.419598,1.213911,-1.074655,-0.493655,-0.618777,2.956923,-1.694394,0.895081,-0.374444,0.769761,0.896513,-0.635384,1.365942,-0.425252,-0.900937,-0.639780,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929
110994,0.049986,-0.543007,2.431874,-0.311952,-0.805988,-0.587013,-0.443149,2.093494,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.324958,-1.396372,-0.370692,2.349369,-1.778847,-0.592261,-3.552328e-16,-1.713636,-0.335981,-1.190102,-1.362990,0.086523,0.115607,-1.615819,0.347414,-0.218574,0.111491,-0.091172,-1.565619,1.219920,0.870335,0.564379,0.100875,0.233184,-0.712243,1.213911,2.132581,-0.493655,-1.476394,0.464332,-0.515582,-1.291095,-0.374444,0.769761,0.896513,-0.635384,-0.732096,-0.425252,-0.900937,1.563037,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929
291457,-0.957891,-0.543007,-1.639185,-0.311952,-0.805988,-0.587013,2.256576,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.495578,0.716142,-0.370692,-0.425646,-0.039750,-0.592261,-5.745391e-01,1.078377,-0.335981,-1.008422,1.709675,0.086523,0.055035,0.785650,-1.092421,-0.564436,-0.288540,0.342236,1.567119,-0.521609,1.218578,-0.331863,0.981360,0.233184,-1.419598,1.213911,-1.074655,2.943773,0.238841,0.464332,-0.515582,-0.926732,-0.374444,0.769761,-1.115433,-0.635384,1.365942,-0.425252,1.109956,-0.639780,-0.139791,0.274741,-0.012298,1.181663,-0.002366,0.452136,0.448929
3095,-0.957891,-0.543007,-0.898992,-0.311952,-0.805988,1.703539,-0.443149,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.607012,0.716142,-0.370692,-0.425646,-1.778847,-0.592261,-3.552328e-16,-0.915918,-0.335981,-1.008422,-1.362990,0.086523,-1.580409,-0.415085,-1.092421,-0.503109,0.156465,0.130960,-0.173291,0.175003,0.522092,-1.228105,1.861846,-0.516858,-0.004888,-0.157794,-0.272846,1.225059,0.238841,-1.197395,0.073825,-0.926732,-0.374444,-1.299104,0.896513,1.573851,-0.732096,-0.425252,-0.900937,1.563037,-0.139791,0.274741,-0.012298,1.181663,-0.002366,0.452136,0.448929


In [ ]:
# Visuallização dos dados no df de teste sem normalização
test.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_03_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1
274442,685895,0,1,3.0,5,3.0,0,0,1,0,0,0,0,0,0,13,1,0,0,0.4,0.2,0.737818,11.0,NaN,0,0,0.0,1,86,3.0,0.316228,0.743838,0.361525,3.316625,0.1,0.1,0.0,1,3,7,3,11,4,14,8,1,0,5,0,1,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
582075,1455073,0,0,1.0,7,0.0,0,1,0,0,0,0,0,0,0,3,0,0,1,0.6,1.2,1.190851,11.0,0.0,9,15,2.0,1,40,2.0,0.547723,1.212529,0.416533,2.645751,0.4,0.4,0.4,2,4,4,1,11,1,7,4,5,0,10,0,1,1,0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
110994,277852,0,2,1.0,11,0.0,0,0,0,1,0,0,0,0,0,12,0,0,1,0.1,0.2,0.894580,4.0,NaN,0,0,0.0,1,65,1.0,0.400000,0.763920,0.379473,3.000000,0.0,0.8,0.7,3,2,8,2,11,5,7,2,2,2,4,0,1,1,0,0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
291457,728292,0,0,1.0,0,0.0,0,0,1,0,0,0,0,0,0,2,1,0,0,0.6,0.2,0.715017,11.0,NaN,0,1,3.0,1,63,3.0,0.316228,0.686312,0.361939,3.316625,0.9,0.3,0.8,2,3,8,1,11,1,17,6,2,2,5,0,1,0,0,1,0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3095,7922,0,0,1.0,2,0.0,0,1,0,0,0,0,0,0,0,13,1,0,0,0.1,0.2,0.894580,6.0,NaN,0,1,0.0,1,9,2.0,0.316228,0.700074,0.381445,3.162278,0.4,0.5,0.6,1,4,7,3,9,2,12,6,0,3,5,0,0,1,1,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


#### Preparar a tabela Dataprep final

In [ ]:
# Trazendo o id e target para a tabela dataprep com os dados normalizados
# Tabela de treino
abt_train = df_train_04.merge(train[['id', 'target']], left_index=True, right_index=True, how='inner')

In [ ]:
abt_train.head(5)

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1,id,target
510263,0.553925,-0.543007,1.321585,-0.311952,-0.805988,-0.587013,2.256576,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.777632,-1.396372,2.697660,-0.425646,-1.083208,-1.086909,0.281831,1.078377,0.594728,0.808380,0.685453,0.086523,-1.459265,-2.816553,-0.096613,-0.616369,1.269533e+00,-0.326028,-0.521373,0.523309,0.522092,-0.331863,-1.660097,-0.516858,-0.712243,1.213911,-0.272846,0.537574,-1.047586,-0.366532,0.073825,-0.198007,-0.374444,0.769761,-1.115433,-0.635384,-0.732096,-0.425252,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1274939,0
566227,-0.957891,-0.543007,0.211296,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.042904,0.716142,-0.370692,-0.425646,-0.039750,-1.086909,-0.684878,-0.915918,-0.335981,0.081659,-1.362990,0.086523,1.024187,0.785650,1.153130,-1.003274,1.266881e+00,-2.828849,1.219037,-1.566526,1.566821,-1.228105,-0.779611,-1.266900,0.702467,0.528059,1.330772,-0.837398,-0.189968,-1.197395,-0.515582,-0.562370,-0.374444,-1.299104,0.896513,1.573851,1.365942,-0.425252,-0.900937,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1415654,0
154123,-0.453952,0.961267,-0.898992,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,0.196743,-1.396372,2.697660,-0.425646,1.003708,-0.344937,-1.029515,1.078377,-0.335981,-1.190102,0.685453,0.086523,0.751613,0.785650,-1.092421,-0.508637,-2.980005e-01,0.130960,1.567119,1.219920,-0.174393,0.564379,0.100875,-0.516858,-1.419598,-0.157794,-1.876463,-0.149912,1.525267,-1.197395,0.073825,1.259443,-0.374444,0.769761,-1.115433,1.573851,1.365942,-0.425252,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,385376,1
500653,1.561802,-0.543007,-0.158800,-0.311952,-0.805988,1.703539,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.042904,0.716142,-0.370692,-0.425646,1.003708,0.397036,-0.702581,1.078377,0.129373,-0.826741,0.685453,0.086523,1.266475,-0.415085,1.158897,4.284008,1.266456e-15,0.544108,0.174791,0.175003,1.218578,1.460621,-0.779611,0.983227,-1.419598,0.528059,0.528963,0.537574,-1.476394,0.464332,0.663231,0.895081,-0.374444,-1.299104,-1.115433,-0.635384,1.365942,-0.425252,-0.900937,-0.63978,7.153559,-3.639785,-0.012298,-0.846265,-0.002366,-2.211725,0.448929,1251174,0
362160,-0.957891,-0.543007,-1.269089,-0.311952,1.240714,-0.587013,-0.443149,-0.47767,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.777632,-1.396372,-0.370692,-0.425646,1.003708,2.128305,1.029377,1.078377,-0.335981,-1.190102,0.685453,0.086523,-0.490113,0.785650,0.347414,-0.116975,-1.882943e-01,0.342236,0.870955,0.871614,0.870335,0.564379,1.861846,-0.516858,-0.712243,-0.843646,1.330772,-1.181140,0.238841,-0.366532,0.073825,0.530718,-0.374444,0.769761,0.896513,-0.635384,-0.732096,-0.425252,1.109956,-0.63978,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,-2.211725,0.448929,904785,0


In [ ]:
# Trazendo o id e target para a tabela dataprep com os dados normalizados
# Tabela de test
abt_test = test_df.merge(test[['id', 'target']], left_index=True, right_index=True, how='inner')

In [ ]:
abt_test.head(5)

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_04_cat,ps_car_06_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_car_05_cat_0.5261814898236643,ps_car_05_cat_1.0,ps_car_07_cat_0.9479847665595149,ps_car_07_cat_1.0,ps_ind_04_cat_0.4173684372663293,ps_ind_04_cat_1.0,ps_car_02_cat_0.8302751137556877,ps_car_02_cat_1.0,ps_car_08_cat_1,id,target
274442,-0.453952,2.465540,0.211296,1.918116,-0.805988,-0.587013,2.256576,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.607012,0.716142,-0.370692,-0.425646,-0.735389,-0.592261,-5.015862e-01,1.078377,-0.335981,-1.190102,-1.362990,0.086523,0.751613,0.785650,-1.092421,-0.308070,-0.298001,0.342236,-1.217537,-1.218220,-1.567364,-1.228105,0.981360,-0.516858,-0.004888,1.213911,1.330772,1.912545,1.096458,-0.366532,-1.694394,-0.926732,-0.374444,0.769761,-1.115433,-0.635384,-0.732096,-0.425252,1.109956,-0.639780,-0.139791,0.274741,-0.012298,1.181663,-0.002366,0.452136,0.448929,685895,0
582075,-0.957891,-0.543007,0.951489,-0.311952,-0.805988,1.703539,-0.443149,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.213525,-1.396372,-0.370692,2.349369,-0.039750,1.880981,9.479614e-01,1.078377,3.852213,1.535100,0.685453,0.086523,-0.641543,-0.415085,2.886394,1.780666,0.956994,-0.576081,-0.173291,-0.173303,-0.174393,-0.331863,1.861846,-2.766985,-1.419598,1.213911,-1.074655,-0.493655,-0.618777,2.956923,-1.694394,0.895081,-0.374444,0.769761,0.896513,-0.635384,1.365942,-0.425252,-0.900937,-0.639780,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,1455073,0
110994,0.049986,-0.543007,2.431874,-0.311952,-0.805988,-0.587013,-0.443149,2.093494,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.324958,-1.396372,-0.370692,2.349369,-1.778847,-0.592261,-3.552328e-16,-1.713636,-0.335981,-1.190102,-1.362990,0.086523,0.115607,-1.615819,0.347414,-0.218574,0.111491,-0.091172,-1.565619,1.219920,0.870335,0.564379,0.100875,0.233184,-0.712243,1.213911,2.132581,-0.493655,-1.476394,0.464332,-0.515582,-1.291095,-0.374444,0.769761,0.896513,-0.635384,-0.732096,-0.425252,-0.900937,1.563037,-0.139791,0.274741,-0.012298,-0.846265,-0.002366,0.452136,0.448929,277852,0
291457,-0.957891,-0.543007,-1.639185,-0.311952,-0.805988,-0.587013,2.256576,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,-1.495578,0.716142,-0.370692,-0.425646,-0.039750,-0.592261,-5.745391e-01,1.078377,-0.335981,-1.008422,1.709675,0.086523,0.055035,0.785650,-1.092421,-0.564436,-0.288540,0.342236,1.567119,-0.521609,1.218578,-0.331863,0.981360,0.233184,-1.419598,1.213911,-1.074655,2.943773,0.238841,0.464332,-0.515582,-0.926732,-0.374444,0.769761,-1.115433,-0.635384,1.365942,-0.425252,1.109956,-0.639780,-0.139791,0.274741,-0.012298,1.181663,-0.002366,0.452136,0.448929,728292,0
3095,-0.957891,-0.543007,-0.898992,-0.311952,-0.805988,1.703539,-0.443149,-0.477670,-0.019446,-0.041262,-0.097325,-0.030227,-0.096992,1.607012,0.716142,-0.370692,-0.425646,-1.778847,-0.592261,-3.552328e-16,-0.915918,-0.335981,-1.008422,-1.362990,0.086523,-1.580409,-0.415085,-1.092421,-0.503109,0.156465,0.130960,-0.173291,0.175003,0.522092,-1.228105,1.861846,-0.516858,-0.004888,-0.157794,-0.272846,1.225059,0.238841,-1.197395,0.073825,-0.926732,-0.374444,-1.299104,0.896513,1.573851,-0.732096,-0.425252,-0.900937,1.563037,-0.139791,0.274741,-0.012298,1.181663,-0.002366,0.452136,0.448929,7922,0


In [ ]:
abt_train.shape

(357127, 62)

In [ ]:
abt_test.shape

(238085, 62)

#### Salvar as tabelas de treino e teste após a preparação de todos os dados
Esses dados serão utilizados para criação do modelo baseline e na para a Feature Selection

In [ ]:
# Salva os dados para serem utilizados na Feature Selection
abt_train.to_csv(dir_base + 'abt_train.csv')
abt_test.to_csv(dir_base + 'abt_test.csv')